## モジュール読み込み

In [1]:
import pandas as pd
import numpy as np
import re
import datetime
from tqdm import tqdm
import seaborn as sns

## 関数

In [2]:
# re_k_results.pyのcsv
df_k = pd.read_csv("/volume_dir/boatrace/results_csv/results.csv")
df_b = pd.read_csv("/volume_dir/boatrace/timetable_csv/timetable.csv")


In [3]:
def merge_preprocess(df_k,df_b):
    drop_col_k = ['boat','st_time']
    df_k.drop(drop_col_k,axis=1,inplace=True)
    drop_col_b = ['date','place','round','position','name','motor']
    df_b.drop(drop_col_b,axis=1,inplace=True)
    # mergeする
    df = pd.merge(df_k,df_b,on=['race_code','number'])
    # 文字を削除
    df = df[~df['in'].astype(str).str.contains('\D')]
    df = df[~df['order'].astype(str).str.contains('\D')]
    list_std = ['win_p','win_p_2','a_win_p','a_win_p_2','motor_p_2','boat_p_2']
    for list in list_std:    
        df_std = df[list].astype(float)
        win_mean=df_std.mean()
        win_std=df_std.std()
        if win_std==0: #もしdf_stdが同じ値だとwin_stdが0になってしまう
            df[list]=50.0
        else:
            df[list] = df_std.apply(lambda x : ((x - win_mean)*10/win_std+50))
    df['date']=df['date'].str.replace(r'[第日]','')
        #roundのRを削除
    df['round']=df['round'].str.replace(r'[R]','')
    # datetimeに変換
    df['race_date'] = pd.to_datetime(df['race_date'])
    # 日だけ抽出
    df['day'] = df['race_date'].dt.day
    # int型に変換
    df['order'] = df['order'].astype('int')
    df['date'] = df['date'].astype('int')
    df['round'] = df['round'].astype('int')
    df['display'] = df['display'].astype('float')
    df['in'] = df['in'].astype('int')
    # race_timeのfloat型に変換
    df['race_time'] = df['race_time'].replace('..','2.0.0').replace('.','2.0.0')
    df['race_time'] = df['race_time'].map(lambda x: float(x[0])*60 + float(x[2:6]))
    # どこかがnanのものを削除
    df.dropna(how='any',inplace=True)
    #　年月日を365日に変換
    df['day_of_year']= df['race_date'].apply(lambda x: x.timetuple().tm_yday)
    # 時系列データとして扱いやすいようにsincosで加工
    encode(df,'day_of_year')
    # 途中からtxtの表記が'琵琶湖'から'びわこ'になっていたため、'びわこ'で統一
    df['place'] = df['place'].map(lambda x: "びわこ" if x == "琵琶湖" else x)
    # indexをrace_dateに変換
    df = df.rename(index=df['race_date'])
    return df

def encode(df, col):
    ###この方法だと場合によって最大値が変化するデータでは正確な値は出ない 例：月の日数が30日や31日の場合がある
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / df[col].max())
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / df[col].max())
    return df

## 実行

In [ ]:
df = merge_preprocess(df_k,df_b)

In [ ]:
#df.to_csv('merged_df.csv',encoding='utf-8',index=False)

## 検証

In [13]:
df = pd.read_csv('/volume_dir/boatrace/merged_df.csv')

In [28]:
df.columns

Index(['race_code', 'date', 'race_date', 'place', 'round', 'order', 'position',
       'number', 'name', 'motor', 'display', 'in', 'race_time', 'type',
       'weather', 'wind', 'wind_strong', 'wave', 'title', 'race_name', 'age',
       'area', 'weight', 'class', 'win_p', 'win_p_2', 'a_win_p', 'a_win_p_2',
       'motor_p_2', 'boat', 'boat_p_2', 'day', 'day_of_year',
       'day_of_year_cos', 'day_of_year_sin'],
      dtype='object')

In [29]:
name_list = df['name'].unique().tolist()

# tqdmを使って進捗を表示する
for name in tqdm(name_list):
    # レース参加者の名前が一致する行を抽出する
    mask = df['name'] == name
    
    # 平均レースタイムを計算する
    mean_time = df.loc[mask, 'race_time'].mean()
    
    # 'race_time_ave'列に平均値を代入する
    df.loc[mask, 'race_time_ave'] = mean_time

100%|██████████| 1946/1946 [02:23<00:00, 13.51it/s]


In [30]:
df['race_time_ave'].value_counts()

113.086494    2162
113.082274    2155
112.265259    2107
112.479113    2097
112.989843    2097
              ... 
114.111765      17
117.943750      16
116.250000      14
118.750000       6
120.000000       1
Name: race_time_ave, Length: 1946, dtype: int64

In [31]:
df.to_csv('merged_df1.csv',encoding='utf-8',index=False)

### race_timeの平均の取り方
+ 直近5レースのrace_timeの平均をとった

In [ ]:
df.set_index('name',inplace=True)

df.set_index('name',inplace=True)
def average(name_list,date,n_samples='all'):
    target_id = df.loc[name_list]
    
    if n_samples == 'all':
        filtered_df = df[df['race_date']<date]
    else:
        filtered_df = df[df['race_date']<date].\
        sort_values('race_date',ascending=False).groupby(level=0).head(n_samples)
    return filtered_df.groupby(level=0)['race_time'].mean()

def merge(df,date,n_samples='all'):
    df = df[df['race_date']==date]
    df.reset_index(inplace=True)
    name_list = df['name']
    merged_df = df.merge(average(name_list,date,n_samples),left_on='name',right_index=True)
    return merged_df

def merge_all(df,n_samples='all'):
    date_list = df['race_date'].unique()
    merged_df = pd.concat([merge(df,date,n_samples) for date in (date_list)])
    return merged_df

df_a = merge_all(df,5)
df_a.rename(columns={'race_time_x':'race_time','race_time_y':'race_time_ave'},inplace=True)

In [ ]:
df_a.to_csv('merged_re_df.csv',encoding='utf-8',index=False)